In [96]:
import numpy as np
from scipy.spatial.distance import hamming
import tensorflow as tf
from collections import OrderedDict
import time

# Crating lookup table for the multiplication

0000000 (0)

1010001 (81)

1110010 (114)

0100011 (35)

0110100 (52)

1100101 (101)

1000110 (70)

0010111 (23)

1101000 (104)

0111001 (57)

0011010 (26)

1001011 (75)

1011100 (92)

0001101 (13)

0101110 (46)

1111111 (127)


In [97]:
table_code_words = np.array([0, 81, 114, 35, 52, 101, 70, 23, 104, 57, 26, 75, 92, 13, 46, 127], dtype=np.uint8)
table_code_words_binary = np.zeros((16,), dtype=np.ndarray)
error_correction = np.zeros((128,), dtype=np.int8)
table_lookup_multiplication = np.zeros((128,), dtype=np.ndarray)

In [98]:
print(table_code_words)
np.save('table_code_words.npy',table_code_words)

[  0  81 114  35  52 101  70  23 104  57  26  75  92  13  46 127]


In [99]:
j = 0
for i in table_code_words:
    table_code_words_binary[j] = [int(bit) for bit in bin(i)[2:].zfill(7)]
    j += 1

print(table_code_words)
print(table_code_words_binary)

[  0  81 114  35  52 101  70  23 104  57  26  75  92  13  46 127]
[list([0, 0, 0, 0, 0, 0, 0]) list([1, 0, 1, 0, 0, 0, 1])
 list([1, 1, 1, 0, 0, 1, 0]) list([0, 1, 0, 0, 0, 1, 1])
 list([0, 1, 1, 0, 1, 0, 0]) list([1, 1, 0, 0, 1, 0, 1])
 list([1, 0, 0, 0, 1, 1, 0]) list([0, 0, 1, 0, 1, 1, 1])
 list([1, 1, 0, 1, 0, 0, 0]) list([0, 1, 1, 1, 0, 0, 1])
 list([0, 0, 1, 1, 0, 1, 0]) list([1, 0, 0, 1, 0, 1, 1])
 list([1, 0, 1, 1, 1, 0, 0]) list([0, 0, 0, 1, 1, 0, 1])
 list([0, 1, 0, 1, 1, 1, 0]) list([1, 1, 1, 1, 1, 1, 1])]


In [100]:
array_j = 0
for i in range(128):

    hd = np.inf
    array_i = [int(bit) for bit in bin(i)[2:].zfill(7)]

    for j in range(16):
        hd_temp = hamming(array_i, table_code_words_binary[j]) * len(array_i)
        if hd_temp < hd:
            hd = hd_temp
            array_j = j - 8
    #         print(hd_temp, array_j)
    #
    # print('------------------------------------')
    error_correction[i] = array_j

print(error_correction)
np.save('table_error_correction.npy', error_correction)

[-8 -8 -8 -5 -8  5 -2 -1 -8  5  2  3  5  5  6  5 -8 -7  2 -1 -4 -1 -1 -1
  2  1  2  2  4  5  2 -1 -8 -5 -5 -5 -4 -3  6 -5  0  1  6 -5  6  5  6  6
 -4  1 -6 -5 -4 -4 -4 -1  1  1  2  1 -4  1  6  7 -8 -7 -2  3 -2 -3 -2 -2
  0  3  3  3  4  5 -2  3 -7 -7 -6 -7  4 -7 -2 -1  4 -7  2  3  4  4  4  7
  0 -3 -6 -5 -3 -3 -2 -3  0  0  0  3  0 -3  6  7 -6 -7 -6 -6 -4 -3 -6  7
  0  1 -6  7  4  7  7  7]


In [101]:
for i in range(128):
    arr = np.zeros((i+1,), dtype=np.int8)

    for j in range(i+1):
        arr[j] = error_correction[j] * error_correction[i]

    table_lookup_multiplication[i] = arr

print(table_lookup_multiplication)
np.save('table_lookup_multiplication.npy', table_lookup_multiplication)

[array([64], dtype=int8) array([64, 64], dtype=int8)
 array([64, 64, 64], dtype=int8) array([40, 40, 40, 25], dtype=int8)
 array([64, 64, 64, 40, 64], dtype=int8)
 array([-40, -40, -40, -25, -40,  25], dtype=int8)
 array([ 16,  16,  16,  10,  16, -10,   4], dtype=int8)
 array([ 8,  8,  8,  5,  8, -5,  2,  1], dtype=int8)
 array([ 64,  64,  64,  40,  64, -40,  16,   8,  64], dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25], dtype=int8)
 array([-16, -16, -16, -10, -16,  10,  -4,  -2, -16,  10,   4], dtype=int8)
 array([-24, -24, -24, -15, -24,  15,  -6,  -3, -24,  15,   6,   9],
       dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25,  10,  15,  25],
       dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25,  10,  15,  25,
         25], dtype=int8)
 array([-48, -48, -48, -30, -48,  30, -12,  -6, -48,  30,  12,  18,  30,
         30,  36], dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25,  10,  15,  25,
       

In [102]:
table = np.load('table_lookup_multiplication.npy', allow_pickle=True)
print(table)

[array([64], dtype=int8) array([64, 64], dtype=int8)
 array([64, 64, 64], dtype=int8) array([40, 40, 40, 25], dtype=int8)
 array([64, 64, 64, 40, 64], dtype=int8)
 array([-40, -40, -40, -25, -40,  25], dtype=int8)
 array([ 16,  16,  16,  10,  16, -10,   4], dtype=int8)
 array([ 8,  8,  8,  5,  8, -5,  2,  1], dtype=int8)
 array([ 64,  64,  64,  40,  64, -40,  16,   8,  64], dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25], dtype=int8)
 array([-16, -16, -16, -10, -16,  10,  -4,  -2, -16,  10,   4], dtype=int8)
 array([-24, -24, -24, -15, -24,  15,  -6,  -3, -24,  15,   6,   9],
       dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25,  10,  15,  25],
       dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25,  10,  15,  25,
         25], dtype=int8)
 array([-48, -48, -48, -30, -48,  30, -12,  -6, -48,  30,  12,  18,  30,
         30,  36], dtype=int8)
 array([-40, -40, -40, -25, -40,  25, -10,  -5, -40,  25,  10,  15,  25,
       

In [103]:
fail = 0
for i in range(128):
    for j in range(128):
        try:
            print("try: ", table[i][j])
        except:
            try:
                print("except: ", table[j][i])
            except:
                print("except: error")
                fail += 1

print(fail)

try:  64
except:  64
except:  64
except:  40
except:  64
except:  -40
except:  16
except:  8
except:  64
except:  -40
except:  -16
except:  -24
except:  -40
except:  -40
except:  -48
except:  -40
except:  64
except:  56
except:  -16
except:  8
except:  32
except:  8
except:  8
except:  8
except:  -16
except:  -8
except:  -16
except:  -16
except:  -32
except:  -40
except:  -16
except:  8
except:  64
except:  40
except:  40
except:  40
except:  32
except:  24
except:  -48
except:  40
except:  0
except:  -8
except:  -48
except:  40
except:  -48
except:  -40
except:  -48
except:  -48
except:  32
except:  -8
except:  48
except:  40
except:  32
except:  32
except:  32
except:  8
except:  -8
except:  -8
except:  -16
except:  -8
except:  32
except:  -8
except:  -48
except:  -56
except:  64
except:  56
except:  16
except:  -24
except:  16
except:  24
except:  16
except:  16
except:  0
except:  -24
except:  -24
except:  -24
except:  -32
except:  -40
except:  16
except:  -24
except:  56
except:  

# Full multiplication table 128 * 128

In [104]:
table_lookup_multiplication_full = np.zeros((128,128), dtype=np.int8)
for i in range(128):
    for j in range(128):
        table_lookup_multiplication_full[i][j] = error_correction[j] * error_correction[i]


print(table_lookup_multiplication_full[127])
np.save('table_lookup_multiplication_full.npy', table_lookup_multiplication_full)

[-56 -56 -56 -35 -56  35 -14  -7 -56  35  14  21  35  35  42  35 -56 -49
  14  -7 -28  -7  -7  -7  14   7  14  14  28  35  14  -7 -56 -35 -35 -35
 -28 -21  42 -35   0   7  42 -35  42  35  42  42 -28   7 -42 -35 -28 -28
 -28  -7   7   7  14   7 -28   7  42  49 -56 -49 -14  21 -14 -21 -14 -14
   0  21  21  21  28  35 -14  21 -49 -49 -42 -49  28 -49 -14  -7  28 -49
  14  21  28  28  28  49   0 -21 -42 -35 -21 -21 -14 -21   0   0   0  21
   0 -21  42  49 -42 -49 -42 -42 -28 -21 -42  49   0   7 -42  49  28  49
  49  49]


# Creating nn models

In [105]:
class Module:
    def __init__(self):
        self.modules = OrderedDict()

    def add_module(self, module, name:str):
        self.modules[name] = module

    def forward(self, input) -> np.ndarray:
        for module in self.modules:
            # print(f"module: {module}")
            input = self.modules[module].forward(input)

        return input

In [106]:
#------------------------------------------------------------------------------
#   FullyConnected class
#------------------------------------------------------------------------------
class FullyConnected(Module):
    def __init__(self, w, b, s_w, s_i, s_o, z_i, z_o, min_T, max_T):
        super(FullyConnected, self).__init__()
        self.W = w
        self.b = b
        self.z_i = z_i
        self.z_o = z_o
        self.m = s_i * s_w / s_o
        self.s_b = s_i * s_w
        self.min_T = min_T
        self.max_T = max_T

    def forward(self, input: np.ndarray) -> np.ndarray:

        output_int8 = np.zeros((input.shape[0], self.W.shape[1]), dtype=np.int8)
        output = np.zeros((input.shape[0], self.W.shape[1]), dtype=np.int32)
        a2 = np.zeros((self.W.shape[1]), dtype=np.int32)

        for i in range(input.shape[0]):
            for k in range(self.W.shape[1]):
                for j in range(input.shape[1]):
                    a2[k] += np.int32(self.W[j][k])
                    output[i][k] += np.int32(input[i][j]) * np.int32(self.W[j][k])

                    # print(f'i = {input[i][j]}, W = {self.W[j][k]}, output_32 = {output[i][k]}, a2={a2[k]}')

                # print('-----------------')
                # print(f'z_o = {self.z_o}, m = {self.m}, o = {output[i][k]}, b = {self.b[k]}, a2 = {a2[k]}, z_i = {self.z_i}, zia2: {-self.z_i*a2[k]}')
                # print(f'output_before_saturate_cast = ',np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k]))

                # output_int8[i][k] = tf.dtypes.saturate_cast(np.round(np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k])), tf.int8)

                output_value = np.round(np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k]), 0)

                if output_value > self.max_T:
                    output_int8[i][k] = np.int8(self.max_T)
                elif output_value < self.min_T:
                    output_int8[i][k] = np.int8(self.min_T)
                else:
                    output_int8[i][k] = np.int8(output_value)

                # print('output_int8: ', output_int8[i][k])
                # print('-----------------')

        # print(f'input: {input} \noutput: {output_int8}\n-----------------\n-----------------')
        # print(f'input: {input.dtype} \noutput2: {output_int8.dtype}\n-----------------')

        return output_int8


#------------------------------------------------------------------------------
#   FullyConnected_encoded class
#------------------------------------------------------------------------------
class FullyConnected_encoded(Module):
    def __init__(self, w, w_encoded, b, s_w, s_i, s_o, z_i, z_o, min_T, max_T):
        super(FullyConnected_encoded, self).__init__()
        self.W = w
        self.W_encoded = w_encoded
        self.b = b
        self.z_i = z_i
        self.z_o = z_o
        self.m = s_i * s_w / s_o
        self.s_b = s_i * s_w
        self.min_T = min_T
        self.max_T = max_T

    def forward(self, input: np.ndarray) -> np.ndarray:

        output_int8 = np.zeros((input.shape[0], self.W.shape[1]), dtype=np.int8)
        output = np.zeros((input.shape[0], self.W.shape[1]), dtype=np.int32)
        a2 = np.zeros((self.W.shape[1]), dtype=np.int32)

        for i in range(input.shape[0]):
            for k in range(self.W.shape[1]):
                for j in range(input.shape[1]):
                    a2[k] += np.int32(self.W[j][k])
                    # output[i][k] += np.int32(input[i][j]) * np.int32(self.W[j][k])

                    try:
                        output[i][k] += np.int32(table_lookup_multiplication[input[i][j]][self.W_encoded[j][k]])
                    except:
                        try:
                            output[i][k] += np.int32(table_lookup_multiplication[self.W_encoded[j][k]][input[i][j]])
                        except:
                            print("except: error")



                    # print(f'i = {input[i][j]}, W = {self.W[j][k]}, output_32 = {output[i][k]}, a2={a2[k]}')

                # print('-----------------')
                # print(f'z_o = {self.z_o}, m = {self.m}, o = {output[i][k]}, b = {self.b[k]}, a2 = {a2[k]}, z_i = {self.z_i}, zia2: {-self.z_i*a2[k]}')
                # print(f'output_before_saturate_cast = ',np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k]))

                # output_int8[i][k] = tf.dtypes.saturate_cast(np.round(np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k])), tf.int8)

                output_value = np.int8(np.round(np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k]), 0))

                if output_value > self.max_T:
                    output_int8[i][k] = np.int8(table_code_words[self.max_T+8])
                elif output_value < self.min_T:
                    output_int8[i][k] = np.int8(table_code_words[self.min_T+8])
                else:
                    output_int8[i][k] = np.int8(table_code_words[output_value+8])

                # print('output_int8: ', output_int8[i][k])
                # print('-----------------')

        # print(f'input: {input} \noutput: {output_int8}\n-----------------\n-----------------')
        # print(f'input: {input.dtype} \noutput2: {output_int8.dtype}\n-----------------')

        return output_int8

#------------------------------------------------------------------------------
#   FullyConnected_encoded_full class
#------------------------------------------------------------------------------
class FullyConnected_encoded_full(Module):
    def __init__(self, w, w_encoded, b, s_w, s_i, s_o, z_i, z_o, min_T, max_T):
        super(FullyConnected_encoded_full, self).__init__()
        self.W = w
        self.W_encoded = w_encoded
        self.b = b
        self.z_i = z_i
        self.z_o = z_o
        self.m = s_i * s_w / s_o
        self.s_b = s_i * s_w
        self.min_T = min_T
        self.max_T = max_T

    def forward(self, input: np.ndarray) -> np.ndarray:

        output_int8 = np.zeros((input.shape[0], self.W.shape[1]), dtype=np.int8)
        output = np.zeros((input.shape[0], self.W.shape[1]), dtype=np.int32)
        a2 = np.zeros((self.W.shape[1]), dtype=np.int32)

        for i in range(input.shape[0]):
            for k in range(self.W.shape[1]):
                for j in range(input.shape[1]):
                    a2[k] += np.int32(self.W[j][k])
                    # output[i][k] += np.int32(input[i][j]) * np.int32(self.W[j][k])
                    output[i][k] += np.int32(table_lookup_multiplication_full[input[i][j]][self.W_encoded[j][k]])




                    # print(f'i = {input[i][j]}, W = {self.W[j][k]}, output_32 = {output[i][k]}, a2={a2[k]}')

                # print('-----------------')
                # print(f'z_o = {self.z_o}, m = {self.m}, o = {output[i][k]}, b = {self.b[k]}, a2 = {a2[k]}, z_i = {self.z_i}, zia2: {-self.z_i*a2[k]}')
                # print(f'output_before_saturate_cast = ',np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k]))

                # output_int8[i][k] = tf.dtypes.saturate_cast(np.round(np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k])), tf.int8)

                output_value = np.int8(np.round(np.int32(self.z_o) + self.m * (-(np.int32(self.z_i)*a2[k]) + output[i][k] + self.b[k]), 0))

                if output_value > self.max_T:
                    output_int8[i][k] = np.int8(table_code_words[self.max_T+8])
                elif output_value < self.min_T:
                    output_int8[i][k] = np.int8(table_code_words[self.min_T+8])
                else:
                    output_int8[i][k] = np.int8(table_code_words[output_value+8])

                # print('output_int8: ', output_int8[i][k])
                # print('-----------------')

        # print(f'input: {input} \noutput: {output_int8}\n-----------------\n-----------------')
        # print(f'input: {input.dtype} \noutput2: {output_int8.dtype}\n-----------------')

        return output_int8



#------------------------------------------------------------------------------
#   Quantize class
#------------------------------------------------------------------------------
class Quantize(Module):
    def __init__(self, s, z_i, z_o, d_type):
        super(Quantize, self).__init__()
        self.z_i = z_i
        self.z_o = z_o
        self.s = s
        self.d_type = d_type

        # print(f'Quantize: z_i: {self.z_i} z_o: {self.z_o} s: {self.s} d_type: {self.d_type}')

    def forward(self, input: np.ndarray) -> np.ndarray:
        # converts from int8 to uint8 and vice versa
        if self.d_type is np.int8:
            arr_q = (input + 128).astype(np.uint8)
        elif self.d_type is np.uint8:
            arr_q = (input - 128).astype(np.int8)
        else:
            raise ValueError(f'input type is not supported: {input.dtype}')

        # print(f'input: {input} \noutput: {arr_q}\n-----------------\n-----------------')
        # print(f'input: {input.dtype} \noutput: {arr_q.dtype}\n-----------------')

        return arr_q


In [107]:
# Calculation of a, b parameters

def calculation_a_b(input_array):
    return np.min(input_array), np.max(input_array)

############################################################################################################

# Weight quantization

def weight_scaling_factor(a, b, min_T, max_T):
    s_a = a / min_T
    s_b = b / max_T

    if s_a > s_b:
        return s_a, a, max_T * s_a
    else:
        return s_b, min_T * s_b, b

def clamp(r,a,b):
    return min(max(r, a), b)

def weight_quan(r, a, b, min_T, s):

    q_value = np.round((clamp(r,a,b) - a) / s) + min_T

    # print(f'q_value: {q_value}')

    z = 0
    r = s * (q_value - z)

    # print(f'r: {r}')

    return q_value

def weight_arr_quan(input_arr, min_T, max_T):

    a, b = calculation_a_b(input_arr)
    s, a, b = weight_scaling_factor(a, b, min_T, max_T)

    out_arr = np.zeros(input_arr.shape, dtype=np.int8).T

    for i in range(input_arr.shape[0]):
        for j in range(input_arr.shape[1]):
            out_arr[j][i] = weight_quan(input_arr[i][j], a, b, min_T, s)

    return s, 0, out_arr

############################################################################################################

# Activation quantization

def activation_scaling_factor(a, b, n):
   return (b - a) / (n - 1)

def activation_scale_zero_point(input_array, n):
    a, b = calculation_a_b(input_array)
    s = activation_scaling_factor(a, b, n)
    q_value = np.round((0 - a) / s) - n/2
    z = q_value - (0 / s)

    print(f's: {s} \nz: {z}')

    return s, z

def activation_quan(r, a, b, n, s, z):

    q_value = np.round((clamp(r,a,b) - a) / s) - n/2

    print(f'q_value: {q_value}')

    r = s * (q_value - z)
    print(f'r: {r}')

    return q_value

############################################################################################################

# Bias quantization

def bias_quan(r, s_w, s_i):
    return np.round(r / (s_i * s_w))

def bias_arr_quan(arr, s_w, s_i):

    arr = np.array([bias_quan(r, s_w, s_i) for r in arr], dtype=np.int32)
    s = s_w * s_i

    return s, 0, arr

In [108]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (eva_images, eva_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images_float32_28x28 = train_images.astype(np.float32) / 255.0
eva_images_float32_28x28 = eva_images.astype(np.float32) / 255.0

# image dimensions (assumed square)
image_size = train_images_float32_28x28.shape[1]
input_size = image_size * image_size
print(f'image_size: {image_size}, input_size: {input_size}')

#reshape data to fit model
train_images_float32 = np.reshape(train_images_float32_28x28, [-1, input_size])
eva_images_float32 = np.reshape(eva_images_float32_28x28, [-1, input_size])


image_size: 28, input_size: 784


# NN float32 model

In [109]:
# Define a simple sequential model
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_dim=input_size),
    tf.keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

# Create a basic model instance
model = create_model()
model.load_weights(r'models/mnist_float_nn_tf/mnist_float_weights')

# Define a new model that outputs the intermediate layer
intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)

# Get the intermediate output for the entire input dataset
intermediate_output_float32 = intermediate_model.predict(train_images_float32)

outputs_float32 = model.predict(train_images_float32)

1875/1875 [==============================] - 1s 547us/step


# NN int4 model

In [110]:
int4_s_w1, int4_z_w1, int4_w1 = weight_arr_quan(model.layers[0].get_weights()[0], -7, 7)
int4_s_i1, int4_z_i1 = activation_scale_zero_point(train_images_float32, 16)
int4_s_b1, int4_z_b1, int4_b1 = bias_arr_quan(model.layers[0].get_weights()[1], int4_s_w1, int4_s_i1)

int4_s_w2, int4_z_w2, int4_w2 = weight_arr_quan(model.layers[1].get_weights()[0], -7, 7)
int4_s_i2, int4_z_i2 = activation_scale_zero_point(intermediate_output_float32, 16)
int4_s_b2, int4_z_b2, int4_b2 = bias_arr_quan(model.layers[1].get_weights()[1], int4_s_w2, int4_s_i2)

int4_s_o1, int4_z_o1 = int4_s_i2, int4_z_i2

int4_s_o2, int4_z_o2 = activation_scale_zero_point(outputs_float32, 16)


model_int4 = Module()
model_int4.add_module(FullyConnected(int4_w1.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4.add_module(FullyConnected(int4_w2.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')

print(int4_w1.T.shape)

s: 0.06666666666666667 
z: -8.0
s: 1.5494242350260417 
z: -8.0
s: 4.425647989908854 
z: 0.0
(784, 20)


# eva encoder

In [111]:
eva_images_int4 = np.zeros(eva_images_float32.shape, dtype=np.int8)
eva_int4_encoded = np.zeros(eva_images_float32.shape, dtype=np.int8)

for i in range(eva_images_float32.shape[0]):
    input_int4 = eva_images_float32[i]
    input_int4 = input_int4 / int4_s_i1 + int4_z_i1

    eva_images_int4[i] = input_int4.astype(np.int8)

    for j in range(eva_images_int4.shape[1]):
        eva_int4_encoded[i][j] = table_code_words[(eva_images_int4[i][j] + 8)]

np.save(r'eva_encoded_int4.npy', eva_int4_encoded)
print(f'{np.expand_dims(eva_images_int4[0], axis=0).astype(np.int8).shape}')
print(f'{np.expand_dims(eva_int4_encoded[0], axis=0).astype(np.int8).shape}')
# print(f'eva_encoded_int4: {eva_int4_encoded[0]}')

(1, 784)
(1, 784)


# weight encoder

In [112]:
int4_w1_encoded = np.zeros(int4_w1.shape, dtype=np.int8)
int4_w2_encoded = np.zeros(int4_w2.shape, dtype=np.int8)

for i in range(int4_w1.shape[0]):
    for j in range(int4_w1.shape[1]):
        int4_w1_encoded[i][j] = table_code_words[(int4_w1[i][j] + 8)]

for i in range(int4_w2.shape[0]):
    for j in range(int4_w2.shape[1]):
        int4_w2_encoded[i][j] = table_code_words[(int4_w2[i][j] + 8)]

# NN int4 encoded model

In [113]:
model_int4_encoded = Module()
model_int4_encoded.add_module(FullyConnected_encoded(int4_w1.T, int4_w1_encoded.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4_encoded.add_module(FullyConnected_encoded(int4_w2.T, int4_w2_encoded.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')

# NN int4 encoded model full table

In [114]:
model_int4_encoded_full = Module()
model_int4_encoded_full.add_module(FullyConnected_encoded_full(int4_w1.T, int4_w1_encoded.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4_encoded_full.add_module(FullyConnected_encoded_full(int4_w2.T, int4_w2_encoded.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')

# Test

In [115]:
dif = np.array([0, 0])

correct_int4 = 0
correct_int4_encoded = 0
correct_int4_encoded_full = 0

mistakes_int4 = np.array([])
mistakes_int4_encoded = np.array([])
mistakes_int4_encoded_full = np.array([])

eva_time_encoded_full = np.array([])
eva_time_encoded = np.array([])
eva_time = np.array([])

start_time = time.time()

for i in range(eva_images_float32.shape[0]):
# for i in range(1):

    if i % 1000 == 0 and i != 0:

        hours, rem = divmod(time.time()-round_time, 3600)
        minutes, seconds = divmod(rem, 60)

        print(f"done {i}")
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        print("acc_int4: ", correct_int4 / i)
        print("acc_int4_encoded: ", correct_int4_encoded / i)
        print("dif: ", dif[0] / i)
        print(f'eva_time: {np.mean(eva_time)}')
        print(f'eva_time_encoded: {np.mean(eva_time_encoded)}')
        print(f'eva_time_encoded_full: {np.mean(eva_time_encoded_full)}')
        print("----------------------------------------------------------------------------")

    if i % 1000 == 0:
        round_time = time.time()

    eva_time1 = time.time()
    output_model_int4 = model_int4.forward(np.expand_dims(eva_images_int4[i], axis=0).astype(np.int8))
    eva_time = np.append(eva_time, time.time() - eva_time1)

    eva_time2 = time.time()
    output_model_int4_encoded = model_int4_encoded.forward(np.expand_dims(eva_int4_encoded[i], axis=0).astype(np.int8))
    eva_time_encoded = np.append(eva_time_encoded, time.time() - eva_time2)

    eva_time3 = time.time()
    output_model_int4_encoded_full = model_int4_encoded_full.forward(np.expand_dims(eva_int4_encoded[i], axis=0).astype(np.int8))
    eva_time_encoded_full = np.append(eva_time_encoded_full, time.time() - eva_time3)

    decoded1 = np.zeros(output_model_int4_encoded.shape)

    for j in range(output_model_int4_encoded.shape[1]):
        decoded1[0][j] = np.where(table_code_words == output_model_int4_encoded[0][j])[0][0] - 8

    decoded2 = np.zeros(output_model_int4_encoded_full.shape)

    for j in range(output_model_int4_encoded_full.shape[1]):
        decoded2[0][j] = np.where(table_code_words == output_model_int4_encoded_full[0][j])[0][0] - 8


    dif[0] += np.sum(np.abs(output_model_int4 - decoded1))
    dif[1] += np.sum(np.abs(output_model_int4 - decoded2))

    answer_int4 = np.argmax(output_model_int4)
    answer_int4_encoded = np.argmax(decoded1)
    answer_int4_encoded_full = np.argmax(decoded2)


    if answer_int4 == eva_labels[i]:
        correct_int4 += 1
    else:
        mistakes_int4 = np.append(mistakes_int4, i)

    if answer_int4_encoded == eva_labels[i]:
        correct_int4_encoded += 1
    else:
        mistakes_int4_encoded = np.append(mistakes_int4_encoded, i)

    if answer_int4_encoded_full == eva_labels[i]:
        correct_int4_encoded_full += 1
    else:
        mistakes_int4_encoded_full = np.append(mistakes_int4_encoded_full, i)


hours, rem = divmod(time.time()-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

np.save('mistakes_encoded_int4_layers2_neurons20-10/dif_0encoded_1encodedfull.npy', dif)
np.save('mistakes_encoded_int4_layers2_neurons20-10/mistakes_int4.npy', mistakes_int4)
np.save('mistakes_encoded_int4_layers2_neurons20-10/mistakes_int4_encoded.npy', mistakes_int4_encoded)
np.save('mistakes_encoded_int4_layers2_neurons20-10/mistakes_int4_encoded_full.npy', mistakes_int4_encoded_full)

mistake_output_1 = np.sum(dif[0]) / eva_images_float32.shape[0]
mistake_value_1 = mistake_output_1 / 10

mistake_output_2 = np.sum(dif[1]) / eva_images_float32.shape[0]
mistake_value_2 = mistake_output_2 / 10

print(f'Mistake1 in output: {mistake_output_1}')
print(f'Mistake1 in value: {mistake_value_1}\n')
print(f'Mistake2 in output: {mistake_output_2}')
print(f'Mistake2 in value: {mistake_value_2}\n')

print('Accuracy float32          : ', model.evaluate(eva_images_float32, eva_labels)[1])
print('Accuracy int4             : ', correct_int4 / eva_images_float32.shape[0])
print('Accuracy int4 encoded     : ', correct_int4_encoded / eva_images_float32.shape[0])
print('Accuracy int4 encoded full: ', correct_int4_encoded_full / eva_images_float32.shape[0])

print(f'eva_time: {np.mean(eva_time)}')
print(f'eva_time_encoded: {np.mean(eva_time_encoded)}')
print(f'eva_time_encoded_full: {np.mean(eva_time_encoded_full)}')

done 1000
00:01:29.62
acc_int4:  0.873
acc_int4_encoded:  0.873
dif:  0.0
eva_time: 0.02490944790840149
eva_time_encoded: 0.032304532527923584
eva_time_encoded_full: 0.03231687116622925
----------------------------------------------------------------------------
done 2000
00:01:30.67
acc_int4:  0.8535
acc_int4_encoded:  0.8535
dif:  0.0
eva_time: 0.024925055384635925
eva_time_encoded: 0.03255127489566803
eva_time_encoded_full: 0.03257195782661438
----------------------------------------------------------------------------
done 3000
00:01:28.64
acc_int4:  0.8513333333333334
acc_int4_encoded:  0.8513333333333334
dif:  0.0
eva_time: 0.02465067195892334
eva_time_encoded: 0.03243563055992126
eva_time_encoded_full: 0.03243564732869466
----------------------------------------------------------------------------
done 4000
00:01:40.14
acc_int4:  0.8535
acc_int4_encoded:  0.8535
dif:  0.0
eva_time: 0.025362350702285767
eva_time_encoded: 0.03345924776792526
eva_time_encoded_full: 0.03329078042507

# NN 200-200-200-10 architecture

In [88]:
import pathlib


# Define a simple sequential model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(200, activation='relu', input_dim=input_size),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

# Create a basic model instance
model = create_model()
model.fit(
    train_images_float32,
    train_labels,
    epochs=10,
    batch_size=64,
    validation_data=(eva_images_float32, eva_labels)
)

tflite_models_dir = pathlib.Path("models")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

float_weights_path = r"./models/mnist_float_nn_tf_layers4_neurons_200-200-200-10/mnist_float_weights"

# Save the model:
float_weights_model_file = tflite_models_dir / "mnist_float_nn_tf_layers4_neurons_200-200-200-10/mnist_float_weights"
float_weights_model_file_index = tflite_models_dir / "mnist_float_nn_tf_layers4_neurons_200-200-200-10/mnist_float_weights.index"
if not float_weights_model_file_index.is_file():
    model.save_weights(float_weights_model_file)
    print("Float weights saved to: ", float_weights_model_file)

Epoch 1/10
938/938 [==============================] - 2s 2ms/step - loss: 0.2265 - accuracy: 0.9311 - val_loss: 0.1091 - val_accuracy: 0.9646
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0919 - accuracy: 0.9713 - val_loss: 0.1098 - val_accuracy: 0.9649
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0644 - accuracy: 0.9798 - val_loss: 0.0938 - val_accuracy: 0.9706
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0481 - accuracy: 0.9848 - val_loss: 0.0837 - val_accuracy: 0.9759
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0387 - accuracy: 0.9876 - val_loss: 0.0738 - val_accuracy: 0.9787
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0327 - accuracy: 0.9893 - val_loss: 0.0787 - val_accuracy: 0.9772
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0288 - accuracy: 0.9906 - val_loss: 0.0781 - val_accuracy: 0.9782
Epoch 

In [89]:
model = create_model()
model.load_weights(float_weights_model_file)

In [90]:
# Define a new model that outputs the intermediate layer
intermediate_model_1 = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
intermediate_model_2 = tf.keras.Model(inputs=model.input, outputs=model.layers[1].output)
intermediate_model_3 = tf.keras.Model(inputs=model.input, outputs=model.layers[2].output)

# Get the intermediate output for the entire input dataset
intermediate_output_1_float32 = intermediate_model_1.predict(train_images_float32)
intermediate_output_2_float32 = intermediate_model_2.predict(train_images_float32)
intermediate_output_3_float32 = intermediate_model_3.predict(train_images_float32)

outputs_float32 = model.predict(train_images_float32)

1875/1875 [==============================] - 2s 1ms/step


In [91]:
int4_s_w1, int4_z_w1, int4_w1 = weight_arr_quan(model.layers[0].get_weights()[0], -7, 7)
int4_s_i1, int4_z_i1 = activation_scale_zero_point(train_images_float32, 16)
int4_s_b1, int4_z_b1, int4_b1 = bias_arr_quan(model.layers[0].get_weights()[1], int4_s_w1, int4_s_i1)

int4_s_w2, int4_z_w2, int4_w2 = weight_arr_quan(model.layers[1].get_weights()[0], -7, 7)
int4_s_i2, int4_z_i2 = activation_scale_zero_point(intermediate_output_1_float32, 16)
int4_s_b2, int4_z_b2, int4_b2 = bias_arr_quan(model.layers[1].get_weights()[1], int4_s_w2, int4_s_i2)

int4_s_w3, int4_z_w3, int4_w3 = weight_arr_quan(model.layers[2].get_weights()[0], -7, 7)
int4_s_i3, int4_z_i3 = activation_scale_zero_point(intermediate_output_2_float32, 16)
int4_s_b3, int4_z_b3, int4_b3 = bias_arr_quan(model.layers[2].get_weights()[1], int4_s_w3, int4_s_i3)

int4_s_w4, int4_z_w4, int4_w4 = weight_arr_quan(model.layers[3].get_weights()[0], -7, 7)
int4_s_i4, int4_z_i4 = activation_scale_zero_point(intermediate_output_3_float32, 16)
int4_s_b4, int4_z_b4, int4_b4 = bias_arr_quan(model.layers[3].get_weights()[1], int4_s_w4, int4_s_i4)

int4_s_o1, int4_z_o1 = int4_s_i2, int4_z_i2
int4_s_o2, int4_z_o2 = int4_s_i3, int4_z_i3
int4_s_o3, int4_z_o3 = int4_s_i4, int4_z_i4
int4_s_o4, int4_z_o4 = activation_scale_zero_point(outputs_float32, 16)

model_int4 = Module()
model_int4.add_module(FullyConnected(int4_w1.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4.add_module(FullyConnected(int4_w2.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')
model_int4.add_module(FullyConnected(int4_w3.T, int4_b3, int4_s_w3, int4_s_i3, int4_s_o3, int4_z_i3, int4_z_o3, -8, 7), 'l3')
model_int4.add_module(FullyConnected(int4_w4.T, int4_b4, int4_s_w4, int4_s_i4, int4_s_o4, int4_z_i4, int4_z_o4, -8, 7), 'l4')

s: 0.06666666666666667 
z: -8.0
s: 0.6302001953125 
z: -8.0
s: 0.9201698303222656 
z: -8.0
s: 1.2737585703531902 
z: -8.0
s: 7.535989379882812 
z: -2.0


In [92]:
int4_w1_encoded = np.zeros(int4_w1.shape, dtype=np.int8)
int4_w2_encoded = np.zeros(int4_w2.shape, dtype=np.int8)
int4_w3_encoded = np.zeros(int4_w3.shape, dtype=np.int8)
int4_w4_encoded = np.zeros(int4_w4.shape, dtype=np.int8)

for i in range(int4_w1.shape[0]):
    for j in range(int4_w1.shape[1]):
        int4_w1_encoded[i][j] = table_code_words[(int4_w1[i][j] + 8)]

for i in range(int4_w2.shape[0]):
    for j in range(int4_w2.shape[1]):
        int4_w2_encoded[i][j] = table_code_words[(int4_w2[i][j] + 8)]

for i in range(int4_w3.shape[0]):
    for j in range(int4_w3.shape[1]):
        int4_w3_encoded[i][j] = table_code_words[(int4_w3[i][j] + 8)]

for i in range(int4_w4.shape[0]):
    for j in range(int4_w4.shape[1]):
        int4_w4_encoded[i][j] = table_code_words[(int4_w4[i][j] + 8)]

In [93]:
model_int4_encoded = Module()
model_int4_encoded.add_module(FullyConnected_encoded(int4_w1.T, int4_w1_encoded.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4_encoded.add_module(FullyConnected_encoded(int4_w2.T, int4_w2_encoded.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')
model_int4_encoded.add_module(FullyConnected_encoded(int4_w3.T, int4_w3_encoded.T, int4_b3, int4_s_w3, int4_s_i3, int4_s_o3, int4_z_i3, int4_z_o3, -8, 7), 'l3')
model_int4_encoded.add_module(FullyConnected_encoded(int4_w4.T, int4_w4_encoded.T, int4_b4, int4_s_w4, int4_s_i4, int4_s_o4, int4_z_i4, int4_z_o4, -8, 7), 'l4')


In [94]:
model_int4_encoded_full = Module()
model_int4_encoded_full.add_module(FullyConnected_encoded_full(int4_w1.T, int4_w1_encoded.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4_encoded_full.add_module(FullyConnected_encoded_full(int4_w2.T, int4_w2_encoded.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')
model_int4_encoded_full.add_module(FullyConnected_encoded_full(int4_w3.T, int4_w3_encoded.T, int4_b3, int4_s_w3, int4_s_i3, int4_s_o3, int4_z_i3, int4_z_o3, -8, 7), 'l3')
model_int4_encoded_full.add_module(FullyConnected_encoded_full(int4_w4.T, int4_w4_encoded.T, int4_b4, int4_s_w4, int4_s_i4, int4_s_o4, int4_z_i4, int4_z_o4, -8, 7), 'l4')


In [95]:
dif = np.array([0, 0])

correct_int4 = 0
correct_int4_encoded = 0
correct_int4_encoded_full = 0

mistakes_int4 = np.array([])
mistakes_int4_encoded = np.array([])
mistakes_int4_encoded_full = np.array([])

eva_time_encoded_full = np.array([])
eva_time_encoded = np.array([])
eva_time = np.array([])

start_time = time.time()

for i in range(eva_images_float32.shape[0]):
# for i in range(1):

    if i % 1000 == 0 and i != 0:

        hours, rem = divmod(time.time()-round_time, 3600)
        minutes, seconds = divmod(rem, 60)

        print(f"done {i}")
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        print("acc_int4: ", correct_int4 / i)
        print("acc_int4_encoded: ", correct_int4_encoded / i)
        print("dif: ", dif[0] / i)
        print(f'eva_time: {np.mean(eva_time)}')
        print(f'eva_time_encoded: {np.mean(eva_time_encoded)}')
        print(f'eva_time_encoded_full: {np.mean(eva_time_encoded_full)}')
        print("----------------------------------------------------------------------------")

    if i % 1000 == 0:
        round_time = time.time()

    eva_time1 = time.time()
    output_model_int4 = model_int4.forward(np.expand_dims(eva_images_int4[i], axis=0).astype(np.int8))
    eva_time = np.append(eva_time, time.time() - eva_time1)

    eva_time2 = time.time()
    output_model_int4_encoded = model_int4_encoded.forward(np.expand_dims(eva_int4_encoded[i], axis=0).astype(np.int8))
    eva_time_encoded = np.append(eva_time_encoded, time.time() - eva_time2)

    eva_time3 = time.time()
    output_model_int4_encoded_full = model_int4_encoded_full.forward(np.expand_dims(eva_int4_encoded[i], axis=0).astype(np.int8))
    eva_time_encoded_full = np.append(eva_time_encoded_full, time.time() - eva_time3)

    decoded1 = np.zeros(output_model_int4_encoded.shape)

    for j in range(output_model_int4_encoded.shape[1]):
        decoded1[0][j] = np.where(table_code_words == output_model_int4_encoded[0][j])[0][0] - 8

    decoded2 = np.zeros(output_model_int4_encoded_full.shape)

    for j in range(output_model_int4_encoded_full.shape[1]):
        decoded2[0][j] = np.where(table_code_words == output_model_int4_encoded_full[0][j])[0][0] - 8


    dif[0] += np.sum(np.abs(output_model_int4 - decoded1))
    dif[1] += np.sum(np.abs(output_model_int4 - decoded2))

    answer_int4 = np.argmax(output_model_int4)
    answer_int4_encoded = np.argmax(decoded1)
    answer_int4_encoded_full = np.argmax(decoded2)


    if answer_int4 == eva_labels[i]:
        correct_int4 += 1
    else:
        mistakes_int4 = np.append(mistakes_int4, i)

    if answer_int4_encoded == eva_labels[i]:
        correct_int4_encoded += 1
    else:
        mistakes_int4_encoded = np.append(mistakes_int4_encoded, i)

    if answer_int4_encoded_full == eva_labels[i]:
        correct_int4_encoded_full += 1
    else:
        mistakes_int4_encoded_full = np.append(mistakes_int4_encoded_full, i)


hours, rem = divmod(time.time()-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

np.save('mistakes_encoded_int4_layers2_neurons200-200-200-10/dif_0encoded_1encodedfull.npy', dif)
np.save('mistakes_encoded_int4_layers2_neurons200-200-200-10/mistakes_int4.npy', mistakes_int4)
np.save('mistakes_encoded_int4_layers2_neurons200-200-200-10/mistakes_int4_encoded.npy', mistakes_int4_encoded)
np.save('mistakes_encoded_int4_layers2_neurons200-200-200-10/mistakes_int4_encoded_full.npy', mistakes_int4_encoded_full)

mistake_output_1 = np.sum(dif[0]) / eva_images_float32.shape[0]
mistake_value_1 = mistake_output_1 / 10

mistake_output_2 = np.sum(dif[1]) / eva_images_float32.shape[0]
mistake_value_2 = mistake_output_2 / 10

print(f'Mistake1 in output: {mistake_output_1}')
print(f'Mistake1 in value: {mistake_value_1}\n')
print(f'Mistake2 in output: {mistake_output_2}')
print(f'Mistake2 in value: {mistake_value_2}\n')

print('Accuracy float32          : ', model.evaluate(eva_images_float32, eva_labels)[1])
print('Accuracy int4             : ', correct_int4 / eva_images_float32.shape[0])
print('Accuracy int4 encoded     : ', correct_int4_encoded / eva_images_float32.shape[0])
print('Accuracy int4 encoded full: ', correct_int4_encoded_full / eva_images_float32.shape[0])

print(f'eva_time: {np.mean(eva_time)}')
print(f'eva_time_encoded: {np.mean(eva_time_encoded)}')
print(f'eva_time_encoded_full: {np.mean(eva_time_encoded_full)}')

done 1000
00:21:45.24
acc_int4:  0.96
acc_int4_encoded:  0.96
dif:  0.0
eva_time: 0.3586824109554291
eva_time_encoded: 0.47297768378257754
eva_time_encoded_full: 0.4734635181427002
----------------------------------------------------------------------------
done 2000
00:21:41.00
acc_int4:  0.943
acc_int4_encoded:  0.943
dif:  0.0
eva_time: 0.3583483741283417
eva_time_encoded: 0.4721818039417267
eva_time_encoded_full: 0.47244298934936524
----------------------------------------------------------------------------
done 3000
00:21:37.44
acc_int4:  0.94
acc_int4_encoded:  0.94
dif:  0.0
eva_time: 0.3580908537705739
eva_time_encoded: 0.47143937810262043
eva_time_encoded_full: 0.4715459376970927
----------------------------------------------------------------------------
done 4000
00:21:39.30
acc_int4:  0.9395
acc_int4_encoded:  0.9395
dif:  0.0
eva_time: 0.3579321005344391
eva_time_encoded: 0.4713199902176857
eva_time_encoded_full: 0.471344836473465
-----------------------------------------

# NN 500-500-500-10 architecture

In [37]:
# Define a simple sequential model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(500, activation='relu', input_dim=input_size),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

# Create a basic model instance
model = create_model()
model.fit(
    train_images_float32,
    train_labels,
    epochs=10,
    batch_size=64,
    validation_data=(eva_images_float32, eva_labels)
)

tflite_models_dir = pathlib.Path("models")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

float_weights_path = r"./models/mnist_float_nn_tf_layers4_neurons_500-500-500-10/mnist_float_weights"

# Save the model:
float_weights_model_file = tflite_models_dir / "mnist_float_nn_tf_layers4_neurons_500-500-500-10/mnist_float_weights"
float_weights_model_file_index = tflite_models_dir / "mnist_float_nn_tf_layers4_neurons_500-500-500-10/mnist_float_weights.index"
if not float_weights_model_file_index.is_file():
    model.save_weights(float_weights_model_file)
    print("Float weights saved to: ", float_weights_model_file)

Epoch 1/10
938/938 [==============================] - 6s 6ms/step - loss: 0.2014 - accuracy: 0.9373 - val_loss: 0.1126 - val_accuracy: 0.9644
Epoch 2/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0841 - accuracy: 0.9738 - val_loss: 0.0900 - val_accuracy: 0.9724
Epoch 3/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0585 - accuracy: 0.9820 - val_loss: 0.1132 - val_accuracy: 0.9649
Epoch 4/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0473 - accuracy: 0.9853 - val_loss: 0.0994 - val_accuracy: 0.9725
Epoch 5/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0386 - accuracy: 0.9878 - val_loss: 0.1146 - val_accuracy: 0.9702
Epoch 6/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0336 - accuracy: 0.9897 - val_loss: 0.0762 - val_accuracy: 0.9794
Epoch 7/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0278 - accuracy: 0.9916 - val_loss: 0.0853 - val_accuracy: 0.9781
Epoch 

In [38]:
model = create_model()
model.load_weights(float_weights_model_file)

In [39]:
# Define a new model that outputs the intermediate layer
intermediate_model_1 = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
intermediate_model_2 = tf.keras.Model(inputs=model.input, outputs=model.layers[1].output)
intermediate_model_3 = tf.keras.Model(inputs=model.input, outputs=model.layers[2].output)

# Get the intermediate output for the entire input dataset
intermediate_output_1_float32 = intermediate_model_1.predict(train_images_float32)
intermediate_output_2_float32 = intermediate_model_2.predict(train_images_float32)
intermediate_output_3_float32 = intermediate_model_3.predict(train_images_float32)

outputs_float32 = model.predict(train_images_float32)

1875/1875 [==============================] - 3s 2ms/step


In [40]:
int4_s_w1, int4_z_w1, int4_w1 = weight_arr_quan(model.layers[0].get_weights()[0], -7, 7)
int4_s_i1, int4_z_i1 = activation_scale_zero_point(train_images_float32, 16)
int4_s_b1, int4_z_b1, int4_b1 = bias_arr_quan(model.layers[0].get_weights()[1], int4_s_w1, int4_s_i1)

int4_s_w2, int4_z_w2, int4_w2 = weight_arr_quan(model.layers[1].get_weights()[0], -7, 7)
int4_s_i2, int4_z_i2 = activation_scale_zero_point(intermediate_output_1_float32, 16)
int4_s_b2, int4_z_b2, int4_b2 = bias_arr_quan(model.layers[1].get_weights()[1], int4_s_w2, int4_s_i2)

int4_s_w3, int4_z_w3, int4_w3 = weight_arr_quan(model.layers[2].get_weights()[0], -7, 7)
int4_s_i3, int4_z_i3 = activation_scale_zero_point(intermediate_output_2_float32, 16)
int4_s_b3, int4_z_b3, int4_b3 = bias_arr_quan(model.layers[2].get_weights()[1], int4_s_w3, int4_s_i3)

int4_s_w4, int4_z_w4, int4_w4 = weight_arr_quan(model.layers[3].get_weights()[0], -7, 7)
int4_s_i4, int4_z_i4 = activation_scale_zero_point(intermediate_output_3_float32, 16)
int4_s_b4, int4_z_b4, int4_b4 = bias_arr_quan(model.layers[3].get_weights()[1], int4_s_w4, int4_s_i4)

int4_s_o1, int4_z_o1 = int4_s_i2, int4_z_i2
int4_s_o2, int4_z_o2 = int4_s_i3, int4_z_i3
int4_s_o3, int4_z_o3 = int4_s_i4, int4_z_i4
int4_s_o4, int4_z_o4 = activation_scale_zero_point(outputs_float32, 16)

model_int4 = Module()
model_int4.add_module(FullyConnected(int4_w1.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4.add_module(FullyConnected(int4_w2.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')
model_int4.add_module(FullyConnected(int4_w3.T, int4_b3, int4_s_w3, int4_s_i3, int4_s_o3, int4_z_i3, int4_z_o3, -8, 7), 'l3')
model_int4.add_module(FullyConnected(int4_w4.T, int4_b4, int4_s_w4, int4_s_i4, int4_s_o4, int4_z_i4, int4_z_o4, -8, 7), 'l4')

s: 0.06666666666666667 
z: -8.0
s: 0.6250583012898763 
z: -8.0
s: 0.966307004292806 
z: -8.0
s: 1.8490086873372396 
z: -8.0
s: 8.219944254557292 
z: -1.0


In [41]:
int4_w1_encoded = np.zeros(int4_w1.shape, dtype=np.int8)
int4_w2_encoded = np.zeros(int4_w2.shape, dtype=np.int8)
int4_w3_encoded = np.zeros(int4_w3.shape, dtype=np.int8)
int4_w4_encoded = np.zeros(int4_w4.shape, dtype=np.int8)

for i in range(int4_w1.shape[0]):
    for j in range(int4_w1.shape[1]):
        int4_w1_encoded[i][j] = table_code_words[(int4_w1[i][j] + 8)]

for i in range(int4_w2.shape[0]):
    for j in range(int4_w2.shape[1]):
        int4_w2_encoded[i][j] = table_code_words[(int4_w2[i][j] + 8)]

for i in range(int4_w3.shape[0]):
    for j in range(int4_w3.shape[1]):
        int4_w3_encoded[i][j] = table_code_words[(int4_w3[i][j] + 8)]

for i in range(int4_w4.shape[0]):
    for j in range(int4_w4.shape[1]):
        int4_w4_encoded[i][j] = table_code_words[(int4_w4[i][j] + 8)]

In [42]:
model_int4_encoded = Module()
model_int4_encoded.add_module(FullyConnected_encoded(int4_w1.T, int4_w1_encoded.T, int4_b1, int4_s_w1, int4_s_i1, int4_s_o1, int4_z_i1, int4_z_o1, -8, 7), 'l1')
model_int4_encoded.add_module(FullyConnected_encoded(int4_w2.T, int4_w2_encoded.T, int4_b2, int4_s_w2, int4_s_i2, int4_s_o2, int4_z_i2, int4_z_o2, -8, 7), 'l2')
model_int4_encoded.add_module(FullyConnected_encoded(int4_w3.T, int4_w3_encoded.T, int4_b3, int4_s_w3, int4_s_i3, int4_s_o3, int4_z_i3, int4_z_o3, -8, 7), 'l3')
model_int4_encoded.add_module(FullyConnected_encoded(int4_w4.T, int4_w4_encoded.T, int4_b4, int4_s_w4, int4_s_i4, int4_s_o4, int4_z_i4, int4_z_o4, -8, 7), 'l4')

In [43]:
dif = np.array([0])

correct_int4 = 0
correct_int4_encoded = 0

mistakes_int4 = np.array([])
mistakes_int4_encoded = np.array([])

eva_time_encoded = np.array([])
eva_time = np.array([])

start_time = time.time()

for i in range(eva_images_float32.shape[0]):
# for i in range(1):

    if i % 1000 == 0 and i != 0:

        hours, rem = divmod(time.time()-round_time, 3600)
        minutes, seconds = divmod(rem, 60)

        print(f"done {i}")
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        print("acc_int4: ", correct_int4 / i)
        print("acc_int4_encoded: ", correct_int4_encoded / i)
        print("dif: ", dif[0] / i)
        print(f'eva_time: {np.mean(eva_time)}')
        print(f'eva_time_encoded: {np.mean(eva_time_encoded)}')
        print("----------------------------------------------------------------------------")

    if i % 1000 == 0:
        round_time = time.time()

    eva_time1 = time.time()
    output_model_int4 = model_int4.forward(np.expand_dims(eva_images_int4[i], axis=0).astype(np.int8))
    eva_time = np.append(eva_time, time.time() - eva_time1)
    eva_time2 = time.time()
    output_model_int4_encoded = model_int4_encoded.forward(np.expand_dims(eva_int4_encoded[i], axis=0).astype(np.int8))
    eva_time_encoded = np.append(eva_time_encoded, time.time() - eva_time2)

    decoded = np.zeros(output_model_int4_encoded.shape)

    for j in range(output_model_int4_encoded.shape[1]):
        decoded[0][j] = np.where(table_code_words == output_model_int4_encoded[0][j])[0][0] - 8


    dif[0] += np.sum(np.abs(output_model_int4 - decoded))

    answer_int4 = np.argmax(output_model_int4)
    answer_int4_encoded = np.argmax(decoded)


    if answer_int4 == eva_labels[i]:
        correct_int4 += 1
    else:
        mistakes_int4 = np.append(mistakes_int4, i)

    if answer_int4_encoded == eva_labels[i]:
        correct_int4_encoded += 1
    else:
        mistakes_int4_encoded = np.append(mistakes_int4_encoded, i)


hours, rem = divmod(time.time()-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

np.save('mistakes_encoded_int4_layers4_neurons500-500-500-10/dif.npy', dif)
np.save('mistakes_encoded_int4_layers4_neurons500-500-500-10/mistakes_int4.npy', mistakes_int4)
np.save('mistakes_encoded_int4_layers4_neurons500-500-500-10/mistakes_int4_encoded.npy', mistakes_int4_encoded)

mistake_output = np.sum(dif) / eva_images_float32.shape[0]
mistake_value = mistake_output / 10

print(f'Mistake in output: {mistake_output}')
print(f'Mistake in value: {mistake_value}\n')

print('Accuracy float32     : ', model.evaluate(eva_images_float32, eva_labels)[1])
print('Accuracy int4        : ', correct_int4 / eva_images_float32.shape[0])
print('Accuracy int4 encoded: ', correct_int4_encoded / eva_images_float32.shape[0])

print(f'eva_time: {np.mean(eva_time)}')
print(f'eva_time_encoded: {np.mean(eva_time_encoded)}')

done 1000
00:51:53.70
acc_int4:  0.961
acc_int4_encoded:  0.961
dif:  0.0
----------------------------------------------------------------------------
done 2000
00:51:31.49
acc_int4:  0.9455
acc_int4_encoded:  0.9455
dif:  0.0
----------------------------------------------------------------------------
done 3000
00:51:31.71
acc_int4:  0.9436666666666667
acc_int4_encoded:  0.9436666666666667
dif:  0.0
----------------------------------------------------------------------------
done 4000
00:51:29.95
acc_int4:  0.94275
acc_int4_encoded:  0.94275
dif:  0.0
----------------------------------------------------------------------------
done 5000
00:51:31.20
acc_int4:  0.9404
acc_int4_encoded:  0.9404
dif:  0.0
----------------------------------------------------------------------------
done 6000
00:51:30.80
acc_int4:  0.9458333333333333
acc_int4_encoded:  0.9458333333333333
dif:  0.0
----------------------------------------------------------------------------
done 7000
00:51:28.27
acc_int4:  0

# Fault attack

In [ ]:
def fault_attack(nn_arrays, n_faults):

    size = 0
    for arr in nn_arrays:
        size += arr.size

    if n_faults >= size:
        nn_arrays *= -1

    elif n_faults > 0:

        index = 0
        arr_n_faults = np.zeros(nn_arrays.shape[0], dtype=np.int64)

        while n_faults != 0:

            n_faults_arr = min(nn_arrays[index].size - arr_n_faults[index], n_faults)

            if(n_faults_arr>0):
                n_faults_arr = np.random.randint(0, n_faults_arr+1)
                n_faults -= n_faults_arr
                arr_n_faults[index] += n_faults_arr


            index += 1
            if index == nn_arrays.shape[0]:
                index = 0

        print(arr_n_faults)

        for i in range(nn_arrays.shape[0]):

            attack = np.random.permutation(nn_arrays[i].size)[:arr_n_faults[i]]

            for j in attack:
                neuron = int(j / nn_arrays[i].shape[1])
                weight = j % nn_arrays[i].shape[1]
                nn_arrays[i][neuron][weight] *= -1